# Assembling a Few-Shot Learning Dataset of Molecules from ChEMBL

Here we describe the procedure used to extract the final dataset. The final dataset was obtained through implementation of four key steps: 

1. Query ChEMBL to obtain initial raw data
2. Clean the data to ensure good quality, and threshold to derive binary classification labels
3. Selection of assays for use in the pretraining, vs. those selected as few-shot testing tasks and for validation.
4. Featurization of the data to prepare suitable input to a range of models

## Setup

Extracting the dataset requires access to a MySQL database server holding the ChEMBL dataset. You can download the data and find instructions on setting this up on https://chembl.gitbook.io/chembl-interface-documentation/downloads.
You will then need to update `fs_mol/preprocessing/utils/config.ini` with the connection information about your MySQL server.

```python
[mysql]
host = # host 
database = # database name 
user = # username
password = # password 
```

Finally, we need to set up a few small bits to run this notebook successfully:

In [1]:
import os
import sys
import json
import pandas as pd
from tqdm import tqdm

# This should be the location of the checkout of the FS-Mol repository:
FS_MOL_CHECKOUT_PATH = os.path.join(os.environ['HOME'], "work", "FS-Mol")
# This should be the where the result of the data extraction will be stored, requiring roughly TODO of space
FS_MOL_RESULT_PATH = "/run/user/1002/tmp/fs_mol"

os.chdir(FS_MOL_CHECKOUT_PATH)
sys.path.insert(0, FS_MOL_CHECKOUT_PATH)
os.makedirs(FS_MOL_RESULT_PATH, exist_ok=True)

## 1. Querying ChEMBL

We query a SQL instance of the full ChEMBL database to obtain the raw data.
This is implemented by the script `fs_mol/preprocessing/query.py`, which takes a list of candidate assays that should be considered (the one we used for the dataset released is stored in `fs_mol/preprocessing/utils/helper_files/assays.jsonl` and is read in as default if an alternative is not passed), and creates one `.csv` file for each assay using a range of fields detailed in `fs_mol/preprocessing/utils/queries.py`.

We take a multiple option approach, as we recognise that not all entries in ChEMBL have complete protein target information. When no protein target information is available, the query is carried out for any other information that may be suitable for characterizing the assay such as the target cell type or tissue.

In [46]:
! python fs_mol/preprocessing/query.py --save-dir {FS_MOL_RESULT_PATH}/raw_data  --assay-list-file fs_mol/preprocessing/utils/helper_files/assays.jsonl

Reading config from /home/hojaechoi/work/FS-Mol/fs_mol/preprocessing/utils/config.ini
Reading config from /home/hojaechoi/work/FS-Mol/fs_mol/preprocessing/utils/config.ini
^C
Traceback (most recent call last):
  File "fs_mol/preprocessing/query.py", line 252, in <module>
    run(args)
  File "fs_mol/preprocessing/query.py", line 208, in run
    summary_filenames,
  File "fs_mol/preprocessing/query.py", line 127, in run_query_on_assay
    rows = cursor.fetchall()
KeyboardInterrupt


In [6]:
from fs_mol.preprocessing.utils.queries import (
    CHEMBL_ASSAY_PROTEIN,
    DISTINCT_TABLES,
    COUNT_QUERIES,
    EXTENDED_SINGLE_ASSAY_NOPROTEIN,
    FIELDNAMES,
    SUMMARY_FIELDNAMES,
    COUNTED_SUMMARY_FIELDNAMES,
    PROTEIN_FIELDS,
    CELL_FIELDS,
)


from fs_mol.preprocessing.query import read_db_config, query_db
db_config = read_db_config(section='sqlite')
import sqlite3 as connector
conn = connector.connect(**db_config)
cursor = conn.cursor()

Reading config from /home/hojaechoi/work/FS-Mol/fs_mol/preprocessing/utils/config.ini


In [ ]:
CHEMBL_ASSAY_PROTEIN = (
    "SELECT s.canonical_smiles AS smiles, act.pchembl_value AS pchembl,"
    " act.standard_value AS standard_value,"
    " act.standard_units AS standard_units,"
    " act.standard_relation AS standard_relation,"
    " act.type AS activity_type,"
    " act.activity_comment AS activity_comment,"
    " a.chembl_id AS chembl_id,"
    " a.assay_type AS assay_type,"
    " a.assay_organism AS organism,"
    " a.confidence_score AS confidence_score,"
    " td.tid AS target_id,"
    " td.pref_name AS target,"
    " tt.target_type AS target_type,"
    " protcls.protein_class_id AS protein_id,"
    " protcls.pref_name AS protein_class_name,"
    " protcls.short_name AS protein_short_name,"
    " protcls.class_level AS protein_class_level,"
    " protcls.protein_class_desc AS protein_class_desc"
    " FROM assays a"
    " JOIN activities act ON a.assay_id = act.assay_id"
    " JOIN compound_structures s ON act.molregno = s.molregno"
    " JOIN target_dictionary td on td.tid = a.tid"
    " JOIN target_components tc on td.tid = tc.tid"
    " JOIN target_type tt on tt.target_type = td.target_type"
    " JOIN component_class compcls on tc.component_id = compcls.component_id"
    " JOIN protein_classification protcls on protcls.protein_class_id = compcls.protein_class_id"
    " AND a.chembl_id = {}"
)

In [3]:
CHEMBL_ASSAY = (
    "SELECT s.canonical_smiles AS smiles, act.pchembl_value AS pchembl,"
    " act.standard_value AS standard_value,"
    " act.standard_units AS standard_units,"
    " act.standard_relation AS standard_relation,"
    " act.type AS activity_type,"
    " act.activity_comment AS activity_comment,"
    " a.chembl_id AS chembl_id,"
    " a.assay_type AS assay_type,"
    " a.assay_organism AS organism,"
    " a.confidence_score AS confidence_score"
    " FROM assays a"
    " JOIN activities act ON a.assay_id = act.assay_id"
    " JOIN compound_structures s ON act.molregno = s.molregno"
    " AND a.chembl_id = {}"
)

#### Load DataFold metadata ( stage chembl_id list)

In [2]:
## load DataFold metadata
import json
import gzip
meta_set_name = 'entire_train_set'  # 'fsmol-0.1'
file_path = f'datasets/{meta_set_name}.json'
with open(file_path, 'r') as file:
    json_obj = json.load(file)

In [3]:
## make fsmol-debug.json
import numpy as np
np.random.seed(0)
outfile_path = 'datasets/fsmol-debug.json'
new_folds = {}
new_folds['train'] = np.random.choice(json_obj['train'], size = 200, replace=False).tolist()
new_folds['valid'] = np.random.choice(json_obj['valid'], size = 3, replace=False).tolist()
new_folds['test'] = np.random.choice(json_obj['test'], size = 3, replace=False).tolist()
with open(outfile_path, 'w') as file:
    json.dump(new_folds, file)


#### Extract Assay annotation df from Chembl sqlite DB

In [8]:
## Extract assay information 
col_names = ["smiles", "pchembl",  "standard_value",  "standard_units",  
            "standard_relation",  "activity_type",  "activity_comment",  
            "chembl_id",  "assay_type",  "organism",  "confidence_score",
            "assay_cell_type", "assay_tissue"]


In [ ]:
# extract assay annotation in each assay
for stage, chembl_ids in json_obj.items():
    for chembl_id in chembl_ids:
        query = EXTENDED_SINGLE_ASSAY_NOPROTEIN
        res = query_db(cursor, chembl_id, query)
        df = pd.DataFrame(res, columns = col_names)
        df.to_csv(f'datasets/{meta_set_name}_annot/{stage}/{chembl_id}.csv', index=None)

In [12]:
import rdkit

In [10]:
# extract 'top units' in each assay
chemble_topUnit_dict = {}
for stage, chembl_ids in json_obj.items():
    pbar = tqdm(chembl_ids)
    for chembl_id in pbar:
        query = EXTENDED_SINGLE_ASSAY_NOPROTEIN
        res = query_db(cursor, chembl_id, query)
        df = pd.DataFrame(res, columns = col_names)
        chemble_topUnit_dict[chembl_id] = df.query('standard_units in ("nM", "uM", "%")')['standard_units'].describe().top

100%|██████████| 157/157 [00:00<00:00, 188.19it/s]


In [11]:
with open('datasets/fs-mol/topUnit.json', 'w') as file:
    json.dump(chemble_topUnit_dict, file)

In [23]:
orig_set = set()
for ids in json_obj.values():
    orig_set.update(set(ids))
queried_set = set(chemble_topUnit_dict.keys())

In [25]:
orig_set.difference(queried_set)

set()

In [6]:
transform22 = lambda x: (x-50)/45+4.5
transform11 = lambda x: 9-np.log10(x)

def scale_label(label, unit):
    if unit == '%':
        return transform22(label)
    elif unit == 'nM':
        return transform11(label)
    else:
        return label
    
data = []
indir = 'datasets/fs-mol'
annodir = 'datasets/entire_train_set_annot'
outdir = 'datasets/fs-mol_converted'
for stage, chembl_ids in json_obj.items():
    for chembl_id in chembl_ids:
        infile_path = os.path.join(indir, stage, f'{chembl_id}.jsonl.gz')
        annofile_path = os.path.join(annodir, stage, f'{chembl_id}.csv')
        break
        with gzip.open(file_path, 'rt') as file:
            for line in file:
                json_obj = json.loads(line.strip())
                data.append(json_obj)

### load jsonl gz file into dataframe

In [28]:
import gzip
import pandas as pd
import json

def load_jsonl_gz_file(file_path, fields = None):
    
    data = []
    with gzip.open(file_path, 'rt') as file:
        for line in file:
            json_obj = json.loads(line.strip())
            if fields is None:
                fields = list(json_obj.keys())
            # del json_obj['fingerprints']
            # del json_obj['graph']
            # del json_obj['descriptors']
            # del json_obj['motifs']
            
            data.append({field: json_obj[field] for field in fields})
    df = pd.DataFrame(data)
    return df


# Example usage
file_path = 'datasets/fs-mol/test/CHEMBL1119333.jsonl.gz'
df = load_jsonl_gz_file(file_path, fields = ['SMILES', 'Assay_ID', 
                                             'RegressionProperty', 'LogRegressionProperty'])

# Perform operations on the DataFrame
print(df.head())  # Display the first few rows of the DataFrame


                                              SMILES       Assay_ID  \
0            CN(C)C(=O)Oc1ccc(-c2cn3ccccc3[n+]2C)cc1  CHEMBL1119333   
1         Cc1cccc2n1cc(-c1ccc(OC(=O)N(C)C)cc1)[n+]2C  CHEMBL1119333   
2        CN(C)C(=O)Oc1ccc(-c2cn3c(Cl)cccc3[n+]2C)cc1  CHEMBL1119333   
3  CN(C)C(=O)Oc1ccc(-c2cn3c(C(F)(F)F)cccc3[n+]2C)cc1  CHEMBL1119333   
4        COc1cccc2n1cc(-c1ccc(OC(=O)N(C)C)cc1)[n+]2C  CHEMBL1119333   

  RegressionProperty LogRegressionProperty  
0            17100.0     4.767003889607846  
1             1300.0     5.886056647693163  
2            12600.0     4.899629454882437  
3            10000.0                   5.0  
4             8100.0     5.091514981121351  


In [17]:
df

,SMILES,Property,Assay_ID,RegressionProperty,LogRegressionProperty,Relation,AssayType
0,CN(C)C(=O)Oc1ccc(-c2cn3ccccc3[n+]2C)cc1,0.0,CHEMBL1119333,17100.0,4.767003889607846,=,B
1,Cc1cccc2n1cc(-c1ccc(OC(=O)N(C)C)cc1)[n+]2C,0.0,CHEMBL1119333,1300.0,5.886056647693163,=,B
2,CN(C)C(=O)Oc1ccc(-c2cn3c(Cl)cccc3[n+]2C)cc1,0.0,CHEMBL1119333,12600.0,4.899629454882437,=,B
3,CN(C)C(=O)Oc1ccc(-c2cn3c(C(F)(F)F)cccc3[n+]2C)cc1,0.0,CHEMBL1119333,10000.0,5.0,=,B
4,COc1cccc2n1cc(-c1ccc(OC(=O)N(C)C)cc1)[n+]2C,0.0,CHEMBL1119333,8100.0,5.091514981121351,=,B
...,...,...,...,...,...,...,...
412,O=C(OC1Cc2c(O)cc(O)cc2OC1c1ccc(O)c(O)c1)c1cc(O...,0.0,CHEMBL1119333,28000.0,4.552841968657781,=,B
413,CC[N+](C)(CC)CCC[n+]1c(-c2ccccc2)c2cc(N)ccc2c2...,0.0,CHEMBL1119333,7140.0,5.146301788223826,=,B
414,COc1ccccc1CN(C)CCCCCC(=O)N(C)CCCCCCCCN(C)C(=O)...,1.0,CHEMBL1119333,104.0,6.982966660701219,=,B
415,Nc1ccc2c(c1)c(-c1ccccc1)[n+](CCCNCCCNCCCNc1c3c...,1.0,CHEMBL1119333,1.49,8.826813731587725,=,B


In [30]:

def get_summ_df(df):
    df['RegressionProperty'] = df['RegressionProperty'].astype(float)
    tmp_df = pd.concat([
    df.groupby('Assay_ID').agg({'RegressionProperty': ('describe')}).droplevel(0, axis=1,),
    df.groupby('Assay_ID').agg({'RegressionProperty': ('median', 'skew')}).droplevel(0, axis=1,),
    ], axis = 1).reset_index()
    tmp_df = tmp_df.assign(IQR=tmp_df['75%'] - tmp_df['25%'])
    return tmp_df

In [31]:
from collections import defaultdict

summ_df_dict = defaultdict(list)
for stage, chembl_ids in json_obj.items():
    for chembl_id in chembl_ids:
        infile_path = os.path.join(indir, stage, f'{chembl_id}.jsonl.gz')
        df = load_jsonl_gz_file(infile_path)
        summ_df_dict[stage].append(get_summ_df(df))

In [34]:
len(summ_df_dict['train'])

26868

In [41]:
summ_df = pd.concat(pd.concat(df_li, ignore_index=True).assign(stage=stage) for stage, df_li in summ_df_dict.items())
summ_df.to_csv('datasets/entire_train_set_annot/assay_reglabel_summary.csv')

In [69]:
orig_df = load_jsonl_gz_file(infile_path)
anno_df = pd.read_csv(annofile_path)

In [77]:
chembl_units_dict = {}
for stage, chembl_ids in json_obj.items():
    for chembl_id in chembl_ids:
        infile_path = os.path.join(indir, stage, f'{chembl_id}.jsonl.gz')
        annofile_path = os.path.join(annodir, stage, f'{chembl_id}.csv')
        anno_df = pd.read_csv(annofile_path)
        chembl_units_dict[chembl_id] = anno_df['standard_units'].unique().tolist()

/home/hojaechoi/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [82]:
pd.DataFrame.from_dict(chembl_units_dict, orient = 'index')[1].unique()

array([nan, None, 'nM', "10'-6M", '%', 'uM', 'ug ml-1', "10'-4/M",
       'ug.mL-1', '10^5/M', '/s', "10'-4umol/L", 'pmol/min/mg protein',
       'mM', 'pM', '%,%', 'nM/s', '/M/s', 'ppm', 'umol/L', 'CFU', 'M',
       's-1', 'Ms-1', '/min', 'PuM', 'mg kg-1', 'ucm/s', 'ug', 'Kg',
       'ug/g', 'mm', 'umol/min/mg'], dtype=object)

In [70]:
orig_df['RegressionProperty'] = orig_df['RegressionProperty'].astype(float)
orig_df['SMILES'] = orig_df['SMILES'].astype(str)

In [71]:
anno_df.dtypes

smiles                object
pchembl              float64
standard_value       float64
standard_units        object
standard_relation     object
activity_type         object
activity_comment     float64
chembl_id             object
assay_type            object
organism             float64
confidence_score       int64
assay_cell_type      float64
assay_tissue         float64
dtype: object

In [72]:
orig_df.dtypes

SMILES                    object
Property                  object
Assay_ID                  object
RegressionProperty       float64
LogRegressionProperty     object
Relation                  object
AssayType                 object
dtype: object

In [75]:
anno_df['standard_units'].unique()

array(['nM'], dtype=object)

In [74]:
orig_df.merge(anno_df, 
              left_on = ('SMILES', 'RegressionProperty', 'Relation'),
              right_on = ('smiles', 'standard_value', 'standard_relation'),
              how = 'left'
              )

,SMILES,Property,Assay_ID,RegressionProperty,LogRegressionProperty,Relation,AssayType,smiles,pchembl,standard_value,standard_units,standard_relation,activity_type,activity_comment,chembl_id,assay_type,organism,confidence_score,assay_cell_type,assay_tissue
0,CN(C)C(=O)Oc1ccc(-c2cn3ccccc3[n+]2C)cc1,0.0,CHEMBL1119333,17100.00,4.767003889607846,=,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cc1cccc2n1cc(-c1ccc(OC(=O)N(C)C)cc1)[n+]2C,0.0,CHEMBL1119333,1300.00,5.886056647693163,=,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CN(C)C(=O)Oc1ccc(-c2cn3c(Cl)cccc3[n+]2C)cc1,0.0,CHEMBL1119333,12600.00,4.899629454882437,=,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CN(C)C(=O)Oc1ccc(-c2cn3c(C(F)(F)F)cccc3[n+]2C)cc1,0.0,CHEMBL1119333,10000.00,5.0,=,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,COc1cccc2n1cc(-c1ccc(OC(=O)N(C)C)cc1)[n+]2C,0.0,CHEMBL1119333,8100.00,5.091514981121351,=,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,O=C(OC1Cc2c(O)cc(O)cc2OC1c1ccc(O)c(O)c1)c1cc(O...,0.0,CHEMBL1119333,28000.00,4.552841968657781,=,B,O=C(OC1Cc2c(O)cc(O)cc2OC1c1ccc(O)c(O)c1)c1cc(O...,4.55,28000.0,nM,=,IC50,NaN,CHEMBL1119333,B,NaN,8.0,NaN,NaN
413,CC[N+](C)(CC)CCC[n+]1c(-c2ccccc2)c2cc(N)ccc2c2...,0.0,CHEMBL1119333,7140.00,5.146301788223826,=,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
414,COc1ccccc1CN(C)CCCCCC(=O)N(C)CCCCCCCCN(C)C(=O)...,1.0,CHEMBL1119333,104.00,6.982966660701219,=,B,COc1ccccc1CN(C)CCCCCC(=O)N(C)CCCCCCCCN(C)C(=O)...,6.98,104.0,nM,=,IC50,NaN,CHEMBL1119333,B,NaN,8.0,NaN,NaN
415,Nc1ccc2c(c1)c(-c1ccccc1)[n+](CCCNCCCNCCCNc1c3c...,1.0,CHEMBL1119333,1.49,8.826813731587725,=,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
anno_df.columns

Index(['smiles', 'pchembl', 'standard_value', 'standard_units',
       'standard_relation', 'activity_type', 'activity_comment', 'chembl_id',
       'assay_type', 'organism', 'confidence_score', 'assay_cell_type',
       'assay_tissue'],
      dtype='object')

In [18]:
import pandas as pd
assay = 'CHEMBL5057874'
query = EXTENDED_SINGLE_ASSAY_NOPROTEIN
res = query_db(cursor, assay, query)
col_names = ["smiles", "pchembl",  "standard_value",  "standard_units",  
             "standard_relation",  "activity_type",  "activity_comment",  
             "chembl_id",  "assay_type",  "organism",  "confidence_score",
             "assay_cell_type", "assay_tissue"]
pd.DataFrame(res, columns = col_names)

,smiles,pchembl,standard_value,standard_units,standard_relation,activity_type,activity_comment,chembl_id,assay_type,organism,confidence_score,assay_cell_type,assay_tissue
0,Cc1cccc(COc2ccc3c(C(F)(F)F)cc(=O)oc3c2)c1,5.85,1400.0,nM,=,IC50,None,CHEMBL5057874,B,Homo sapiens,9,LNCaP,Prostate gland
1,O=c1cc(C(F)(F)F)c2ccc(OCc3ccccc3F)cc2o1,6.04,920.0,nM,=,IC50,None,CHEMBL5057874,B,Homo sapiens,9,LNCaP,Prostate gland
2,O=c1cc(C(F)(F)F)c2ccc(OCc3c(F)cccc3Cl)cc2o1,6.11,780.0,nM,=,IC50,None,CHEMBL5057874,B,Homo sapiens,9,LNCaP,Prostate gland
3,O=c1cc(C(F)(F)F)c2ccc(OCc3ccccc3Cl)cc2o1,6.39,410.0,nM,=,IC50,None,CHEMBL5057874,B,Homo sapiens,9,LNCaP,Prostate gland
4,O=c1cc(C(F)(F)F)c2ccc(OCc3cc4c(cc3Cl)OCO4)cc2o1,5.83,1470.0,nM,=,IC50,None,CHEMBL5057874,B,Homo sapiens,9,LNCaP,Prostate gland
5,COc1ccc([N+](=O)[O-])cc1COc1ccc2c(C(F)(F)F)cc(...,NaN,50000.0,nM,>,IC50,None,CHEMBL5057874,B,Homo sapiens,9,LNCaP,Prostate gland
6,CCOC(=O)C(CC)Oc1ccc2c(C(F)(F)F)cc(=O)oc2c1,NaN,50000.0,nM,>,IC50,None,CHEMBL5057874,B,Homo sapiens,9,LNCaP,Prostate gland
7,COC(=O)C(CC(C)C)Oc1ccc2c(C(F)(F)F)cc(=O)oc2c1,NaN,50000.0,nM,>,IC50,None,CHEMBL5057874,B,Homo sapiens,9,LNCaP,Prostate gland
8,O=C(Oc1ccc2c(C(F)(F)F)cc(=O)oc2c1)C1CCCCC1,NaN,50000.0,nM,>,IC50,None,CHEMBL5057874,B,Homo sapiens,9,LNCaP,Prostate gland
9,Cn1cc(C(=O)Nc2ccc3c(C(F)(F)F)cc(=O)oc3c2)cn1,NaN,50000.0,nM,>,IC50,None,CHEMBL5057874,B,Homo sapiens,9,LNCaP,Prostate gland


In [11]:
assay = 'CHEMBL5057874'
query = CHEMBL_ASSAY_PROTEIN
query_db(cursor, assay, query)

[('Cc1cccc(COc2ccc3c(C(F)(F)F)cc(=O)oc3c2)c1',
  5.85,
  'IC50',
  1400,
  'nM',
  '=',
  None,
  'CHEMBL5057874',
  'B',
  'Homo sapiens',
  9,
  56,
  'Androgen Receptor',
  'SINGLE PROTEIN',
  378,
  'Nuclear hormone receptor subfamily 3 group C member 4',
  'NR3C4',
  5,
  'transcription factor  nuclear receptor  nr3  nr3c  nr3c4'),
 ('O=c1cc(C(F)(F)F)c2ccc(OCc3ccccc3F)cc2o1',
  6.04,
  'IC50',
  920,
  'nM',
  '=',
  None,
  'CHEMBL5057874',
  'B',
  'Homo sapiens',
  9,
  56,
  'Androgen Receptor',
  'SINGLE PROTEIN',
  378,
  'Nuclear hormone receptor subfamily 3 group C member 4',
  'NR3C4',
  5,
  'transcription factor  nuclear receptor  nr3  nr3c  nr3c4'),
 ('O=c1cc(C(F)(F)F)c2ccc(OCc3c(F)cccc3Cl)cc2o1',
  6.11,
  'IC50',
  780,
  'nM',
  '=',
  None,
  'CHEMBL5057874',
  'B',
  'Homo sapiens',
  9,
  56,
  'Androgen Receptor',
  'SINGLE PROTEIN',
  378,
  'Nuclear hormone receptor subfamily 3 group C member 4',
  'NR3C4',
  5,
  'transcription factor  nuclear receptor  nr3  

As a result of this raw data extraction, we obtained 36,093 separate raw assay files as `.csv`s, from a list of 36,160. Not all initially identified assays are able to return fields for any query option.

In [10]:
with open(os.path.join(FS_MOL_CHECKOUT_PATH, "fs_mol/preprocessing/utils/helper_files/assays.jsonl"), "r") as jf:
    assays = json.load(jf)
len(assays["assays"])

36160


### Initial List of Assays
Our initial query of ChEMBL selects only those assays that contain more than 32 datapoints. We accessed CHEMBL27 and selected all assays with more than 32 measurements. We record the assay ids and confidence scores, where confidence reflects the level of information about the target protein in the assay: '9' is a known single protein target, '0' is completely unknown, for instance it could be as broad as an entire tissue.

To regenerate this list (after changing criteria, for example), you can run `fs_mol/preprocessing/initial_query.py`:

In [11]:
! python fs_mol/preprocessing/initial_query.py 

Reading config from /home/hojaechoi/work/FS-Mol/fs_mol/preprocessing/utils/config.ini
Reading config from /home/hojaechoi/work/FS-Mol/fs_mol/preprocessing/utils/config.ini
no such table: confidence_score_lookup
Traceback (most recent call last):
  File "fs_mol/preprocessing/initial_query.py", line 170, in <module>
    run()
  File "fs_mol/preprocessing/initial_query.py", line 166, in run
    recreate_assay_list_file(base_output_dir, assay_list_file)
  File "fs_mol/preprocessing/initial_query.py", line 142, in recreate_assay_list_file
    pd.read_csv(os.path.join(base_output_dir, f"assays_{cs}.csv")).chembl_id.values
  File "/anaconda/envs/fsmol/lib/python3.7/site-packages/pandas/util/_decorators.py", line 311, in wrapper
    return func(*args, **kwargs)
  File "/anaconda/envs/fsmol/lib/python3.7/site-packages/pandas/io/parsers/readers.py", line 586, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/anaconda/envs/fsmol/lib/python3.7/site-packages/pandas/io/parsers/readers.

The script outputs the assay list to the default destination of `config.ini`'s `[initialquery]` section


## 2. Cleaning

The cleaning procedure takes place in three keys stages, detailed in `fs_mol/preprocessing/clean.py`:

1. Assays are selected to proceed to the next stage only if they reflect activity or inhibition measurements with units of "%", "uM" or "nM".
2. SMILES are standardized, and XC50 (IC50 or EC50) measurements are converted to -log10([C]/NM) prior to thresholding. This step also de-duplicates measurements where applicable.
3. A final (optional) thresholding step is applied.

### Standardization

The standardization procedure for SMILES is as follows: 

- Remove salts
- Disconnect any metallo-organic complexes
- Make certain the correct ion is present
- Choose the largest fragment if the SMILES string represents disconnected components
- Remove excess charges
- Choose the canonical tautomer

After this procedure, molecules are rejected if they have a molecular weight > 900 Da, and exact SMILES-value duplicate pairs are dropped within an assay. 

**De-duplication** of SMILES then accepts a degree of variation in the measured value for the same SMILES -- if a SMILES value is repeated in a dataframe, we accept measurements where the standard value measured is within the same order of magnitude, to fairly capture measurement noise. We reject all measurements for that SMILES if that is not the case. While this may reject stereoisomers with profoundly different chemical behaviors, we wish to remove erroneous measurements of other molecules. 

### Thresholding

As part of cleaning the data, we automatically derive active/inactive labels from the activity data. Our thresholding proceeds via a automated procedure that attempts to adapt flexibly to each assay to ensure that we do not discount a number of measurements due to overly rigid thresholding rules. 

We take the median value of an assay's activity measurements, and use this as a threshold provided it is in the range 5 $\le$ median(pXC) $\le$ 7 for enzymes, or 4 $\le$ median(pXC) $\le$ 6 for all other assays. If the median is outside this range, we select PKX = 5.0 as our fixed threshold. 

With this threshold we are able to derive a binary activity label.

Overall, the cleaning can be applied to the extracted data as follows, the final directory will be "cleaned", but a custom suffix can be used.

In [ ]:
! python fs_mol/preprocessing/clean.py {FS_MOL_RESULT_PATH} --input-dir raw_data


# 3. Assay Selection for train-valid-test split

Our assay selection proceeds via examining the final sizes of the assays and their associated protein information. We begin with a list of 27004 assays for which cleaning did not result in removal of all data. Not all assays have available protein information. 

In [ ]:
import os
import pandas as pd

In [ ]:
df = pd.read_csv(os.path.join(FS_MOL_CHECKOUT_PATH, "datasets/targets/target_info.csv"))

print(f"We have {df.cleaned_size.sum()} measurements from our first pass of cleaning (cleaning_failed == False)")

In [ ]:
df = pd.concat(
    [
        df.loc[df['target_id'].notna()].astype({"target_id": int}).astype({"target_id": str}),
        df.loc[df['target_id'].isna()]
    ],
    ignore_index=True
)

# first select out assays that are very small
df = df[df.cleaned_size>=32]
print(f"We have {len(df[df.target_id.notna()].target_id.unique())} unique known targets")

EC numbers were assigned by additionally querying ChEMBL for the component and type synonyms from the component synonyms table. Where the type synonym was "EC_number" we are able to assign an EC number to the target. The `EC_super_class_name` is applied from a dictionary mapping from `EC_number`: 
```python
EC_super_class_dict = {'1':'oxidoreductase',
                       "2":'transferase',
                       "3":'hydrolase',
                       "4":'lyase',
                       "5":'isomerase',
                       "6":'ligase',
                       "7":'translocase'}
```

However, there are a number of targets for which this classification is uncertain, and we wish to ensure that our final targets have a confident classification by EC class. The `protein_class_desc` is cleaned to define a `protein_family` and `protein_super_family`. `EC_name` is taken from the `component_synonym`. `reliable_target_EC` is `True` if only one EC number is present for the target (some have a list of conflicting values in the database), and `reliable_target_EC_super` is True if `EC_super_class` is single only. The same approach is taken for target protein descriptions: `reliable_target_protein_desc == True` reflects only single `protein_class_desc` entries (that is, non-type entries), `reliable_target_protein_super == True` is the same for `protein_super_family`.

This information is included in our target info csvs for completeness. 

To select test tasks, we require that they only have well known target ids, and since we also wish to categorise by EC number, we will select those for which a good EC number can be obtained. 

We first extract everything that cannot be included as a few-shot test task, which involves the cases of:
- having no good EC number for the overall class (NaN or EC number super class considered unreliable -- `reliable_target_EC_super == False`). 
- no single target ID available (eg. non-single-protein measurements)

In [ ]:
possible_test = df[df.target_id.notna()]
possible_test = possible_test[possible_test.reliable_target_EC_super.notna()]
possible_test = possible_test[possible_test.reliable_target_EC_super == True]

print(f"Prior to filtering we have: {len(possible_test)} assays with well known EC super classes")
print(f"This consists of {len(possible_test.target_id.unique())} targets with {possible_test.cleaned_size.sum()} individual measurement points.")

We make further stringent requirements here on the test tasks: they must be less than 5000 datapoints to avoid high-throughput screens, as these are generally considered noisy and not in keeping with the QSAR data considered here. 

In [ ]:
best = possible_test.loc[
    (possible_test["cleaned_size"] >= 128) &
    (possible_test["confidence"] >= 8) &
    (possible_test["percentage_pos"] <= 70) &
    (possible_test["percentage_pos"] >= 30) &
    (possible_test["cleaned_size"] <= 5000)
]

print(f"We have {len(set(possible_test.target_id.unique()))} possible test targets")

We would like 200 final few-shot tasks, but we may not be able to achieve this without impoverishing the training set. 

How many of each EC class would this represent, to maintain the current proportions in the data? 

In [ ]:
best.EC_super_class.value_counts() * 200/ best.EC_super_class.value_counts().sum()

In [ ]:
from collections import defaultdict
required_target_numbers = {"EC_super_class": [str(x) for x in range(1, 8)], "target_count": [10, 150, 30, 3, 1, 3, 2]}
ids = defaultdict(set)
test_ids = set()
for c, target_count in zip(required_target_numbers["EC_super_class"],required_target_numbers["target_count"]):
    ids[c] = set(best[best.EC_super_class == c].target_id.value_counts().tail(target_count).index)
    test_ids = test_ids.union(ids[c])

In [ ]:
print(f"We identify a total {len(test_ids)} that may be used in the testing set of tasks")

### Training set

We can assemble the training set from all that remains now that we have selected our target protein IDs. It should be composed of all 'good' protein measurements with known targets and well known EC classes, as well as everything else where these values may be uncertain (for instance, the cases of non-enzymatic proteins where there is no such thing as an EC number).

We then also remove non-protein target measurements (no target id), and suggest these are only to be used as part of an extended training set.

We supply final tables of the train, valid and test set assays.

We note that test assays are required to have a confidence score of 8 or 9, where this reflects a single protein target.

In [ ]:
pd.read_csv(os.path.join(FS_MOL_CHECKOUT_PATH, "datasets/targets/train_proteins.csv"))

In [ ]:
pd.read_csv(os.path.join(FS_MOL_CHECKOUT_PATH, "datasets/targets/test_proteins.csv"))

In [ ]:
pd.read_csv(os.path.join(FS_MOL_CHECKOUT_PATH, "datasets/targets/valid_proteins.csv"))

In practice, the validation set chooses only those tasks for which EC super class is 2 as we want to reduce the time taken by validation steps, and note that majority of tasks in testing are associated with kinases.

# 4. Featurization

In featurization we take the SMILES string (here termed 'canonical' following the careful cleaning) and use it to create rdkit mol objects, from which further featurization can proceed. This takes place in the `featurize.py`. 

The final featurized files include the SMILES string, but also the ECFP fingerprints, standard physico-chemical descriptors from rdkit, and a graph featurization. The graph featurization relies on `metadata.pkl.gz` as it is created by a set of featurizers with fixed vocabularies to maintain consistent featurization across all assays.


In [ ]:
! python fs_mol/preprocessing/featurize.py {FS_MOL_RESULT_PATH}/cleaned {FS_MOL_RESULT_PATH}/processed --balance-limits 30.0 70.0